# CS 4700 - Homework 5: Supervised Learning
<h3 style="color: red;">
    Due: Wednesday, April 17, 1:24pm on CMS
    <br><br>
    Late submission: Friday, April 19, 1:24pm on CMS (50% penalty)
</h3>

Which other students did you interact with on this assignment? Provide their NetID(s) below consistent with the [homework policy](http://www.cs.cornell.edu/courses/cs4700/2019sp/).

Homework 5 will provide you with programming practice in the supervised learning environment. You will be implementing algorithms in Python 3, so please ensure your Jupyter Notebook environment is running the Python 3 kernel. Start by running the cell below. These are the **only** `import` statements allowed in this assignment. **Importing additional libraries will result in an automatic 0!**

Throughout the assignment, you will find sections labeled as **<span style="color: red;">Task</span>**. These sections mark the parts that you will need to work on. The other, un-marked sections simply provide information on the topics for reference.

Lastly, most of your implementations for this homework will need to use functions from the `math` module. For those unfamiliar with the functions offered by this module, please refer to its documentation here: [https://docs.python.org/3/library/math.html](https://docs.python.org/3/library/math.html)

In [1]:
import math    # copysign, exp, inf, log, pow
import random  # shuffle, uniform
import sys     # version_info

assert sys.version_info >= (3,), 'You must use Python 3 for this assignment!'

### Part 1: Logistic Regression

<h3 style="text-align: right; margin-top: -1em;">20 points</h3>

---

Recall the **perceptron linear classifier** from lecture. The perceptron is a **single-layer neural network** that uses a hard threshold, which can cause issues for more complex data sets. One specific downside of this classifier is that it always outputs a confident result of 0 or 1; however, there are many cases where we would instead like to return a granular output, such as a probability. We can fix these issues by replacing the hard threshold function with a soft threshold function. One such function is the **logistic function**: $g \left( z \right) = \frac{1}{1 + e^{-z}}$. The figure below shows a comparison between the two types of threshold functions.

![](hard_vs_soft_threshold.png)

Utilizing the logistic function as our threshold for a single-layer neural network results in the **logistic regression classifier**:

$$
h_\overline{w} \left( \overline{x} \right) = g \left( \overline{w} \cdot \overline{x} \right) = \frac{1}{1 + e^{-\overline{w} \cdot \overline{x}}}
$$

In order for this model to learn the appropriate set of weights, we use the gradient descent method to minimize the error that our model makes. Given that the derivative of the logistic function is $g^\prime \left( z \right) = g \left( z \right) \times \left( 1 - g \left(z \right) \right)$, we can compute the gradient update for each weight (and for some learning rate $\alpha$) as:

$$
w_i \leftarrow w_i + \alpha \times \left( y - g \left( \overline{w} \cdot \overline{x} \right) \right) \times g^\prime \left(\overline{w} \cdot \overline{x} \right) \times x_i
$$

Or equivalenty, in terms of only the logistic function, $g \left( z \right)$:

$$
w_i \leftarrow w_i + \alpha \times \left( y - g \left( \overline{w} \cdot \overline{x} \right) \right) \times g \left(\overline{w} \cdot \overline{x} \right) \times \left( 1 - g \left( \overline{w} \cdot \overline{x} \right) \right) \times x_i
$$

Recall that the weights have an explicit "bias" term as $w_0$. This term has a corresponding entry in each example as a dummy value of 1 (i.e., $x_{i0} = 1$ for all entries $i$ in the data set). The figure below shows an abstract representation of our classifier.

![](logistic_regression.png)

<h2 style="color: red;">Task:</h2>

Implement the logistic regression classifier, using the pseudocode below as your guide. To help, you will need to implement two additional helper functions. `logistic` should implement the logistic function (refer to the previous formula). `dot_prod` should calculate the dot product of the two input vectors (leave the `assert` statement).

---

**Note, the pseudocode below is *slightly* different from what you saw in lecture. Here, we specify the stopping condition for the classifier as just iterating over the data set some number of times. This information is passed in as input to the classifier as `epochs`.**

---

$
\text{Inputs:} \\
D - \text{data set, a list of } \left( \overline{x},y \right) \text{, where } \overline{x} \in \mathbb{R}^n+1 \text{ and } y \in \{0,1\} \\
\alpha - \text{learning rate} \\
\text{epochs} - \text{total number of iterations} \\
\ \\
\overline{\underline{\textbf{logistic_learn} \left( D, \alpha, \text{epochs} \right)}} \\
\quad \overline{w} \leftarrow \overline{0} \text{ of size } n+1 \\
\quad \textbf{while} \text{ epochs} > 0 \\
\qquad \textbf{for each} \left( \overline{x},y \right) \in D \\
\qquad \quad p \leftarrow g \left( \overline{w} \cdot \overline{x} \right) \\
\qquad \quad \textbf{for each} \text{ feature } i \\
\qquad \qquad w_i \leftarrow w_i + \alpha \times \left( y - p \right) \times p \times \left( 1 - p \right) \times x_i \\
\qquad \text{epochs} \leftarrow \text{epochs} - 1 \\
\quad \textbf{return } \overline{w}
$

In [2]:
"""
Returns the value of the logistic function evaluated at z
"""
def logistic(z):
    return 1/(1 + math.exp(-z))

"""
Returns the dot product of xs and ys
"""
def dot_prod(xs, ys):
    assert len(xs) == len(ys), 'Vector inputs must be of same size!'
    sum = 0
    for i in range(len(xs)):
        sum += xs[i] * ys[i]
    return sum
"""
Returns logistic regression coefficient vector by performing logistic regression on data data_set, with learning rate
learning_rate, for epochs iterations
"""
def logistic_learn(data_set, learning_rate, epochs):
    n = len(data_set[0][0]) - 1
    w = list(0 for q in range(n+1))
    while epochs > 0:
        for (x, y) in data_set:
            p = logistic(dot_prod(w,x))
            for i in range(n+1):
                w[i] = w[i] + learning_rate * (y - p) * p * (1-p) * x[i]
        epochs = epochs - 1
    return w

For a sanity check, we have included some test cases that you can run your implementation against. The cell below will pass in a data set containing the 4 cases for the logical **AND** function. The logistic regression classifier should be able to learn the appropriate weights so that the 4 predictions are close to the actual labels. Try changing the learning rate and the number of epochs to get more accurate predictions. Your predictions should be within 0.1 of the true label for each sample.

In [3]:
and_set = [([1, 0, 0], 0), ([1, 0, 1], 0), ([1, 1, 0], 0), ([1, 1, 1], 1)]
weights = logistic_learn(and_set, .99, 30000)
for (x, y) in and_set:
    pred = logistic(dot_prod(weights, x))
    print('{} AND {} is {}, you predicted {}'.format(x[1], x[2], y, pred))
    assert abs(pred - y) < 0.1, 'Your prediction diverged too much from the true label!'

0 AND 0 is 0, you predicted 9.9819531064928e-07
0 AND 1 is 0, you predicted 0.009353054511490051
1 AND 0 is 0, you predicted 0.009354047995585923
1 AND 1 is 1, you predicted 0.9889270199750348


Next, let's ensure that your classifier is also able to learn the weights for the logical **OR** function.

In [4]:
or_set = [([1, 0, 0], 0), ([1, 0, 1], 1), ([1, 1, 0], 1), ([1, 1, 1], 1)]
weights = logistic_learn(or_set, 0.5, 10000)
for (x, y) in or_set:
    pred = logistic(dot_prod(weights, x))
    print('{} OR {} is {}, you predicted {}'.format(x[1], x[2], y, pred))
    assert abs(pred - y) < 0.1, 'Your prediction diverged too much from the true label!'

0 OR 0 is 0, you predicted 0.02331033960673716
0 OR 1 is 1, you predicted 0.9853372900246228
1 OR 0 is 1, you predicted 0.9853342752458599
1 OR 1 is 1, you predicted 0.9999947138561152


The logistic regression classifier is still constrained in what functions it can represent. For example, it still fails to find weights for non-linearly separable data sets. The common example of such a data set is **XOR**. Run the cell below to verify for yourself that our logistic regression classifier is unable to produce correct weights that approximate the logical XOR function.

In [5]:
xor_set = [([1, 0, 0], 0), ([1, 0, 1], 1), ([1, 1, 0], 1), ([1, 1, 1], 0)]
weights = logistic_learn(xor_set, 0.5, 10000)
for (x, y) in xor_set:
    pred = logistic(dot_prod(weights, x))
    print('{} XOR {} is {}, you predicted {}'.format(x[1], x[2], y, pred))

0 XOR 0 is 0, you predicted 0.51610600358629
0 XOR 1 is 1, you predicted 0.5
1 XOR 0 is 1, you predicted 0.4838939964137099
1 XOR 1 is 0, you predicted 0.46782138179306076


### Part 2: Deep Neural Networks

<h3 style="text-align: right; margin-top: -1em;">40 points</h3>

---

To alleviate our issue of non-linearity, we need to replace our shallow, single-layer model. Both the perceptron and logistic regression classifiers have a single input layer and a single output node. But, to capture the non-linear relationship in data, we need additional layers! We refer to these, in general, as **hidden layers**. Combining this deep model of multiple layers with a soft threshold function (**activation function**) results in a **deep (multi-layered) neural network** (**DNN**). For simplicity, we will that each layer is fully connected to all nodes in the layer above it, allowing them to be represented as directed acyclic graphs of **units** or **neurons**. Another characteristic of DNNs is that they allow us to have multiple outputs, which is different from the classifiers we've explored so far. As a consequence, we now treat a data set $D$ to consist of $\left(\overline{x},\overline{y}\right)$, meaning both the data and label are vectors.

The figure below shows an example of a DNN with 1 input layer (2 input units), 1 hidden layer (2 hidden units), and 1 output unit.

![](deep_neural_network.png)

When an input is passed into a DNN, the signals are passed along the network by following the connections between the units, starting at the input layer and ending in the output layer. This process is refered to as **forward propagation**. In order for the DNN to learn, it must update its weights across its multiple layers. The common approach for DNN learning is called **back-propagation**, which relies on the gradient descent method.

The code cell below defines the `DNNUnit` class, which will represent a unit in a DNN. Creating an actual DNN is handled by the function `create_DNN`. Read through and understand the code below. **Of important note is the way the bias unit is handled: every unit in the network (except those in the input layer) has the bias unit as the first input/weight.** Run the cell to include these definitions to the environment.

As an example, we can define the network from the figure above (assuming the logistic function is the activation function) as the following:
```python
logistic_deriv = lambda z : logistic(z) * (1 - logistic(z))
figure_network = create_DNN(2, [2], 1, logistic, logistic_deriv)
```

In [6]:
"""
A unit of a neural network.

Inputs:
  activation - the activation function to be used
  deriv      - the derivative of the activation function
  inputs     - a list of units that feed a signal to this unit
  weights    - a list of weights that correspond to each input signal
"""
class DNNUnit:
    def __init__(self, activation, deriv, inputs = None, weights = None):
        self.activation = activation
        self.deriv = deriv
        self.inputs = inputs or []
        self.weights = weights or []
        self.value = None


"""
Creates a deep, multi-layered neural network. Includes a bias unit of value 1
that is connected to every unit except those in the input layer. Initializes
all weights to a random real number between -0.5 and 0.5, inclusive.

Inputs:
    input_layer_size   - the number of input units for the network
    hidden_layer_sizes - a list with the number of hidden units in each hidden layer
    output_layer_size  - the number of output units for the network
    activation         - the activation function to be used
    deriv              - the derivative of the activation function
"""
def create_DNN(input_layer_size, hidden_layer_sizes, output_layer_size, activation, deriv):
    layer_sizes = [input_layer_size] + hidden_layer_sizes + [output_layer_size]
    network = [[DNNUnit(activation, deriv) for _ in range(s)] for s in layer_sizes]
    dummy_unit = DNNUnit(activation, deriv)
    dummy_unit.value = 1.0
    
    # create weighted links
    for k in range(1, len(layer_sizes)):
        for unit in network[k]:
            unit.inputs.append(dummy_unit)
            unit.weights.append(random.uniform(-0.5, 0.5))
            for input_unit in network[k - 1]:
                unit.inputs.append(input_unit)
                unit.weights.append(random.uniform(-0.5, 0.5))
    
    return network

It is worthwhile to explore the interconnectedness between our above implementation of a DNN and its mathematical representation. The table below shows the attributes of a unit $i$ in a DNN, how to access it in code (let the variable be called `unit`), and its mathematical representation:

| Description | Code | Math |
| ----------- | ---- | ---- |
| Activation function | `unit.activation` | $g\left(z\right)$ |
| Derivative of activation function | `unit.deriv` | $g^\prime\left(z\right)$ |
| Units that connect to $i$ | `unit.inputs` | $\overline{s}_i$ |
| Weights for those units in $\overline{s}_i$ | `unit.weights` | $\overline{v}_i$ |
| Value of the unit $i$ | `unit.value` | $a_i$ |

Typically, when discussing about a unit $i$ in layer $k$, we also need to talk about the units that connect to $i$, the units that $i$ connects to, and their associated attributes. The table below summarizes the specific relationships we need for the propagation algorithms:

| Description | Math |
| ----------- | ---- |
| Values for those units in $\overline{s}_i$ | $\overline{p}_i$ |
| Units in layer $k+1$ that $i$ connects to | $\overline{t}_i$ |
| Weights between $i$ and those units in $\overline{t}_i$ | $\overline{u}_i$ |
| Error for $i$ | $\Delta_i$ |
| Errors of those units in $\overline{t}_i$ | $\overline{q}_i$ |

To make the above information slightly more concrete, let's look at a brief example. Consider the figure below. Note that unit 0 refers to the bias unit.

![](dnn_example.png)

For the figure above, we calculate the relevant information for unit 3 in the table below.

| Math | Value |
| ---- | ----- |
| $\overline{s}_3$ | $\left<\text{Unit 0, Unit 1, Unit 2}\right>$ |
| $\overline{v}_3$ | $\left<w_{0,3}\text{, }w_{1,3}\text{, }w_{2,3}\right>$ |
| $\overline{p}_3$ | $\left<a_0\text{, }a_1\text{, }a_2\right>$ |
| $\overline{t}_3$ | $\left<\text{Unit 4, Unit 5, Unit 6}\right>$ |
| $\overline{u}_3$ | $\left<w_{3,4}\text{, }w_{3,5}\text{, }w_{3,6}\right>$ |
| $\overline{q}_3$ | $\left<\Delta_4\text{, }\Delta_5\text{, }\Delta_6\right>$ |

<h2 style="color: red;">Task:</h2>

Implement the propagation algorithms, `forward_prop` and `back_prop`, using the pseudocode below as a guide.

$
\text{Inputs:} \\
\text{DNN} - \text{current state of the deep neural network,} \\
\quad \text{where each layer } k \text{ consists of units and each unit } i \text{ has:} \\
\qquad \bullet \text{activation function } g \\
\qquad \bullet \text{derivative of activation function } g^\prime \\
\qquad \bullet \text{vector of inputs } \overline{s}_i \\
\qquad \bullet \text{vector of weights } \overline{v}_i \\
\qquad \bullet \text{value } a_i \\
\overline{x} - \text{current vector sample to be forward propagated} \\
\ \\
\overline{\underline{\textbf{forward_prop} \left( \text{DNN, } \overline{x} \right)}} \\
\quad \textbf{for each} \text{ unit } i \in \text{input layer} \\
\qquad a_i \leftarrow x_i \\
\quad \textbf{for each} \text{ layer } k \in \text{DNN starting at second layer to output layer} \\
\qquad \textbf{for each} \text{ unit } i \in k \\
\qquad \quad \overline{p}_i \leftarrow \text{vector of values for units in } \overline{s}_i \\
\qquad \quad a_i \leftarrow g \left( \overline{v}_i \cdot \overline{p}_i \right) \\
$

---

$
\text{Inputs:} \\
\text{DNN} - \text{current state of the deep neural network,} \\
\quad \text{where each layer } k \text{ consists of units and each unit } i \text{ has:} \\
\qquad \bullet \text{activation function } g \\
\qquad \bullet \text{derivative of activation function } g^\prime \\
\qquad \bullet \text{vector of inputs } \overline{s}_i \\
\qquad \bullet \text{vector of weights } \overline{v}_i \\
\qquad \bullet \text{value } a_i \\
D - \text{data set, a list of} \left( \overline{x} , \overline{y} \right), \text{where } \overline{x} \in \mathbb{R}^{n} \text{ and } \overline{y} \in \mathbb{R}^{d} \\
\quad \left( \text{meaning } n \text{ features and } d \text{ number of outputs} \right) \\
\alpha - \text{learning rate} \\
\text{epochs} - \text{total number of iterations} \\
\ \\
\overline{\underline{\textbf{back_prop} \left( \text{DNN,} D, \alpha, \text{epochs} \right)}} \\
\quad \Delta \leftarrow \text{vector of errors, initialized with units in DNN as keys and 0 for values} \\
\quad \textbf{while } \text{epochs} > 0 \\
\qquad \textbf{for each} \left( \overline{x} , \overline{y} \right) \in D \\
\qquad \quad \textbf{forward_prop} \left( \text{DNN, } \overline{x} \right) \\
\qquad \quad \textbf{for each} \text{ unit } i \in \text{output layer} \\
\qquad \qquad \overline{p}_i \leftarrow \text{vector of values for units in } \overline{s}_i \\
\qquad \qquad \Delta_i \leftarrow g^\prime \left( \overline{v}_i \cdot \overline{p}_i \right) \times \left( y_i - a_i \right) \\
\qquad \quad \textbf{for each} \text{ layer } k \text{ starting from last hidden layer to one above input layer} \\
\qquad \qquad \textbf{for each} \text{ unit } i \in k \\
\qquad \qquad \quad \overline{p}_i \leftarrow \text{vector of values for units in } \overline{s}_i \\
\qquad \qquad \quad \overline{t}_i \leftarrow \text{vector of units in layer } k+1 \text{ that } i \text{ connects to} \\
\qquad \qquad \quad \overline{u}_i \leftarrow \text{vector of weights between } i \text{ and those units in } \overline{t}_i \\
\qquad \qquad \quad \overline{q}_i \leftarrow \text{vector of errors of those units in } \overline{t}_i \\
\qquad \qquad \quad \Delta_i \leftarrow g^\prime \left( \overline{v}_i \cdot \overline{p}_i \right) \times \left( \overline{u}_i \cdot \overline{q}_i \right) \\
\qquad \quad \textbf{for each} \text{ weight } w_{ij} \text{ in DNN} \\
\qquad \qquad w_{ij} \leftarrow w_{ij} + \alpha \times a_i \times \Delta_j \\
\qquad \text{epochs} \leftarrow \text{epochs} - 1
$

In [65]:
"""
Performs the forward propagation algorithm given above on DNN network and vector x
"""
def forward_prop(network, x):
    count = -1
    for i in network[0]:
        count = count + 1
        i.value = x[count]
    for k in network[1:len(network)]:
        for i in k:
            p = list()
            for j in range(0, len(i.inputs)):
                p.append(i.inputs[j].value)
            i.value = i.activation(dot_prod(i.weights, p))
            
"""
Performs the backward propagation algorithm given above on DNN network, with data data_set and learning rate learning_rate, for
epochs iterations
"""
def back_prop(network, data_set, learning_rate, epochs):
    delta = {}
    for k in range(0, len(network)):
        for unit in network[k]:
            delta[unit] = 0
    while epochs > 0:
        for (x, y) in data_set:
            forward_prop(network, x)
            count = -1
            for i in network[len(network) - 1]:
                count = count + 1
                p = list()
                for j in range(0, len(i.inputs)):
                    p.append(i.inputs[j].value)
                delta[i] = i.deriv(dot_prod(i.weights, p)) * (y[count] - i.value)
            for k in reversed(range(1, len(network) - 1)):
                count = -1
                for i in network[k]:
                    count = count + 1
                    p = list()
                    for j in range(0, len(i.inputs)):
                        p.append(i.inputs[j].value)
                    t = list()
                    for j in range(1, len(network[k + 1])):
                        t.append(network[k + 1][j])
                    u = list()
                    for g in t:
                        u.append(g.weights[count + 1])
                    q = list()
                    for r in t:
                        q.append(delta[r])
                    delta[i] = i.deriv(dot_prod(i.weights, p)) * dot_prod(u, q)
            for k in range(1, len(network)):
                for j in network[k]:
                    count = -1
                    for i in j.inputs:
                        count = count + 1
                        j.weights[count] = j.weights[count] + learning_rate * i.value * delta[j]
        epochs = epochs - 1
        

<h2 style="color: red;">Task:</h2>

If you have a working implementation of the algorithms, the DNN should be able to learn the **2-bit adder** function. For those unfamiliar, the following table summarizes this function's behavior:

| $x_1$ | $x_2$ | $y_1$ | $y_2$ |
| --- | --- | --- | --- |
| 0 | 0 | 0 | 0 |
| 0 | 1 | 0 | 1 |
| 1 | 0 | 0 | 1 |
| 1 | 1 | 1 | 0 |

In short, the 2-bit adder has 2 outputs, where $y_1$ represents logical **AND** (linearly separable) and $y_2$ represents logical **XOR** (non-linearly separable). Run the code cell below multiple times to ensure your implementation is able to learn the 2-bit adder function. It is possible that the DNN will not approximate the function within an acceptable error-bound, even though your implementation is correct. **Why does this possibility exist?** Write your reasoning in the raw cell below.

In [77]:
data_set = [([0,0],[0,0]), ([0,1],[0,1]), ([1,0],[0,1]), ([1,1],[1,0])]
test_set = [[0,0], [0,1], [1,0], [1,1]]
correct_out = [[0,0], [0,1], [0,1], [1,0]]
logistic_deriv = lambda z : logistic(z) * (1 - logistic(z))
network = create_DNN(2, [2,2], 2, logistic, logistic_deriv)

back_prop(network, data_set, 0.5, 10000)
for t in range(len(test_set)):
    print('Input =', test_set[t])
    print('True Output =', correct_out[t])
    forward_prop(network, test_set[t])
    preds = [node.value for node in network[-1]]
    print('Prediction =', preds)
    devs = [abs(preds[p] - correct_out[t][p]) for p in range(len(preds))]
    for d in devs:
        assert d <= 0.02, 'Your prediction diverged too much from the true output!'
    print()

Input = [0, 0]
True Output = [0, 0]
Prediction = [3.985743142435295e-06, 0.016219097655980366]

Input = [0, 1]
True Output = [0, 1]
Prediction = [0.014475035462051937, 0.9782474439519478]


AssertionError: Your prediction diverged too much from the true output!

<h2 style="color: red;">Task:</h2>

There are many types of activation functions that are used by DNNs. Implement the following set of activation functions and their derivatives. **Do not use the trigonometric functions from the `math` module in your implementation!** Their mathematical definitions are given as:

$$
\begin{align}
    \text{tanh} \left( z \right) &= \frac{2}{1 + e^{-2z}} - 1 &
    \frac{d}{dz} \text{tanh} &= 1 - \text{tanh} \left( z \right)^2 \\
    \text{ReLU} \left( z \right) &=
    \begin{cases}
        z & \text{if } z > 0 \\
        0 & \text{otherwise}
    \end{cases} &
    \frac{d}{dz} \text{ReLU} &= 0.5 + 0.5 \times \text{sgn} \left( z \right) \\
    \text{LeakyReLU} \left( z \right) &=
    \begin{cases}
        z & \text{if } z > 0 \\
        0.01z & \text{otherwise}
    \end{cases} &
    \frac{d}{dz} \text{LeakyReLU} &= 0.505 + 0.495 \times \text{sgn} \left( z \right) \\
    \text{SmoothReLU} \left( z \right) &= \log \left( 1 + e^z \right) &
    \frac{d}{dz} \text{SmoothReLU} &= \frac{1}{1 + e^{-z}} \\
\end{align}
$$

For those unfamiliar, $\text{sgn}\left(z\right)$ refers to the sign function, which is defined as:

$$
\text{sgn} \left( z \right) =
\begin{cases}
    1 & \text{if } z > 0 \\
    0 & \text{else if } z = 0 \\
    -1 & \text{otherwise}
\end{cases}
$$

*Side note:* strictly speaking, the derivates for ReLU and LeakyReLU are undefined for 0. To avoid variations in how to handle this special case, we went ahead and defined the derivates as being the midpoint of the lower and upper values.

In [31]:
"""
Returns the value of the tanh function evaluated at z
"""
def tanh(z):
    return 2/(1 + math.exp(-2 * z)) - 1

"""
Returns the derivative of the tanh function evaluated at z
"""
def tanh_deriv(z):
    return 1 - math.pow(tanh(z), 2)
    
"""
Returns the value of the ReLU function evaluated at z
"""
def relu(z):
    if z > 0:
        return z
    return 0

"""
Returns the derivative of the ReLU function evaluated at z for z nonzero, .5 otherwise
"""
def relu_deriv(z):
    if z > 0:
        return 1
    if z < 0:
        return 0
    return .5

"""
Returns the value of the LeakyReLU function evaluated at z
"""
def leaky_relu(z):
    if z > 0:
        return z
    return .01 * z

"""
Returns the derivative of the ReLU function evaluated at z for z nonzero, .505 otherwise
"""
def leaky_relu_deriv(z):
    if z > 0:
        return 1
    if z < 0:
        return .01
    return .505

"""
Returns the value of the SmoothReLU function evaluated at z
"""
def smooth_relu(z):
    return math.log(1 + math.exp(z))

"""
Returns the derivative of the SmoothReLU function evaluated at z
"""
def smooth_relu_deriv(z):
    return 1/(1 + math.exp(-z))


<h2 style="color: red;">Task:</h2>

Now, let's compare the performance of the activation functions. To do this, we will use a different function for our DNN to approximate. Play around with the configurable variable values below and see how these **hyperparameters** (meaning parameters for the DNN hypothesis class) affect the performance for each activation function. **Which activation function(s) seem to work better for this problem?** Enter your response in the raw cell below.

In [68]:
# configurable variables
# play around with these values and see how it affects the error rate
learning_rate = 0.5
num_epochs = 5000
error_bound = 0.2

# code to test performance
# do not modify
data_set = [([p,q,r,s],[int(p and q), int(q or r), int(r <= s), int(s != p)]) for p in [0,1] for q in [0,1] for r in [0,1] for s in [0,1]]
random.shuffle(data_set)
training_set = data_set[0:12]
logistic_deriv = lambda z : logistic(z) * (1 - logistic(z))
func_names = ["Logistic", "Hypertangent", "ReLU", "Leaky ReLU", "Smooth ReLU"]
act_funcs = [logistic, tanh, relu, leaky_relu, smooth_relu]
deriv_funcs = [logistic_deriv, tanh_deriv, relu_deriv, leaky_relu_deriv, smooth_relu_deriv]
r = random.getstate()
for (n, a, d) in zip(func_names, act_funcs, deriv_funcs):
    random.setstate(r)
    network = create_DNN(4, [8,8], 4, a, d)
    back_prop(network, data_set, learning_rate, num_epochs)
    test_set = [x for (x,y) in data_set[12:]]
    correct_out = [y for (x,y) in data_set[12:]]
    num_misses = 0
    for t in range(len(test_set)):
        forward_prop(network, test_set[t])
        preds = [unit.value for unit in network[-1]]
        devs = [abs(preds[p] - correct_out[t][p]) for p in range(len(preds))]
        for d in devs:
            if d > error_bound:
                num_misses += 1
    print(n, 'error rate:', num_misses / 16.0 * 100.0, '%')

Logistic error rate: 0.0 %
Hypertangent error rate: 12.5 %
ReLU error rate: 31.25 %
Leaky ReLU error rate: 0.0 %
Smooth ReLU error rate: 0.0 %


### Part 3: Naïve Bayes

<h3 style="text-align: right; margin-top: -1em;">30 points</h3>

---

Let's turn our attention to the matter of probabilities. Part 1 mentioned how the logistic function has the nice feature of returning outputs that can be interpreted as probabilities. Let's continue with this train of thought. For this section, we will use $i$ to index over the $N$ samples in the data set, while $j$ will be used to index over the $n$ features in a particular sample $\overline{x}_i$.

In a supervised learning setting, we are interested in estimating $P\left(Y \mid X\right)$, since we are given the data and want to predict the label. This is difficult to estimate, however, in a high dimensional setting. We could rely on Bayes' rule to help, but we end up with another issue when estimating $P\left(X \mid Y\right)$. To make some progress, we take a huge leap of faith by making the **naïve Bayes assumption**, which states that the $n$ features in a sample are independent given the label: $P\left(\overline{x} \mid y\right) = \prod_{j=1}^{n}P\left(x_j \mid y\right)$.

Taking all of this into account, we end up with the following **naïve Bayes classifier**:

$$
\begin{align}
h\left(\overline{x}\right) &= \underset{y}{\text{argmax }} P\left(y \mid \overline{x}\right) \\
&= \underset{y}{\text{argmax }} \frac{P\left(\overline{x} \mid y\right) P\left(y\right)}{P\left(\overline{x}\right)} & \text{Bayes' rule} \\
&= \underset{y}{\text{argmax }} P\left(\overline{x} \mid y\right) P\left(y\right) & P\left(\overline{x}\right) \text{does not depend on } y \\
&= \underset{y}{\text{argmax }} P\left(y\right) \prod_{j=1}^{n} P\left(x_j \mid y\right) & \text{naïve Bayes assumption} \\
&= \underset{y}{\text{argmax }} \log\left(P\left(y\right)\right) + \sum_{j=1}^{n} \log\left(P\left(x_j \mid y\right)\right) & \log\text{is monotonic}
\end{align}
$$

Estimating $P\left(y\right)$ is usually straightforward. We can estimate the probability of the label $y$ having value $c$ with the following (where $I\left(\cdot\right)$ is the indicator function, also defined below):

$$
P\left(y=c\right) = \frac{1}{N} \sum_{i=1}^{N} I\left(y_i=c\right) \\
I\left(x\right) =
\begin{cases}
    1 & \text{if } x \text{ is true} \\
    0 & \text{otherwise}
\end{cases}
$$

How about estimating $P\left(x_j \mid y\right)$? This highly depends on what kind of features the data set contains. For this assignment, we will assume that we are dealing with **multinomial features**, meaning the features in the data represent counts. Thus, we have for any specific sample $\overline{x}_i$:

$$
x_{ij} \in \{0,1,2,\dots\} \text{ and } m_i = \sum_{j=1}^{n} x_{ij}
$$

Note how $m_i$ can vary between samples, since not every sample's features will add up to the same sum. Under this assumption, we can precisely calculate the probability of a sample $\overline{x}$ having the label value $c$ as:

$$
P\left(\overline{x}\mid y=c\right) = \frac{\left(\sum_{j=1}^{n}x_j\right)!}{\prod_{j=1}^{n}x_j!} \prod_{j=1}^{n} \left(\theta_{jc}\right)^{x_j}
$$

We use $\theta_{jc}$ as a parameter to refer to the probability of selecting feature $x_j$ and specify that $\sum_{j=1}^{n}\theta_{jc}=1$. While the above formula might seem intimidating, it does simplify when considered in the argmax setting of our classifier. This is because we can safely ignore the fraction as it does not depend on the label value $c$. We can estimate the $\theta_{jc}$ parameter using the following equation (where $k$ is a smoothing parameter):

$$
\theta_{jc} = \frac{\sum_{i=1}^{N} I\left(y_i=c\right)x_{ij}+k}{\sum_{i=1}^{N} I\left(y_i=c\right)m_i+k\times n}
$$

Finally, combining all of this together, we have that:

$$
\begin{align}
\underset{c}{\text{argmax }}P\left(y=c\mid\overline{x}\right)
&\propto\underset{c}{\text{argmax }}P\left(y=c\right)\prod_{j=1}^{n}\left(\theta_{jc}\right)^{x_j} \\
&\propto\underset{c}{\text{argmax }}\log\left(P\left(y=c\right)\right)+\sum_{j=1}^{n}\log\left(\left(\theta_{jc}\right)^{x_j}\right) \\
&\propto\underset{c}{\text{argmax }}\log\left(P\left(y=c\right)\right)+\sum_{j=1}^{n}x_j\log\left(\theta_{jc}\right)
\end{align}
$$

Therefore, for the naïve Bayes classifier (under a multinomial distribution) to make a prediction for a sample $\overline{x}$, it would iterate over all label values to find the value $c$ that maximizes the above formula.

<h2 style="color: red;">Task:</h2>

Implement the naïve Bayes classifier, `naive_bayes`, and its helper functions below. `label_est` should estimate $P\left(y=c\right)$. `theta_est` should estimate $\theta_{jc}$. You may assume the label space $\mathcal{Y}$ is binary, i.e., $y\in\{0,1\}$.

In [146]:
"""
Returns the estimated probability that a generic observation in data_set pertains to target_label
"""
def label_est(data_set, target_label):
    sum = 0
    for i in data_set:
        if i[1] == target_label:
            sum = sum + 1
    return sum / len(data_set)

"""
Returns the paramter theta, using label target_label, attribute j, and smoothing parameter smooth, for the Naïve Bayes classifier
created with data_set
"""
def theta_est(data_set, target_label, j, smooth):
    num = 0
    den = 0
    for i in range(0, len(data_set)):
        if data_set[i][1] == target_label:
            num = num + data_set[i][0][j]
            m_i = 0
            for g in range(0, len(data_set[0][0])):
                m_i = m_i + data_set[i][0][g]
            den = den + m_i
    num = num + smooth
    den = den + smooth * len(data_set[0][0])
    return(num / den)

"""
Returns the Naïve Bayes classifier (created with data_set) prediction of the label that most likely pertains to observation x,
using smoothing parameter smooth
"""
def naive_bayes(data_set, x, smooth):
    max = -math.inf
    argmax = 0
    for c in range(0, 2):
        result = math.log(label_est(data_set, c))
        for j in range(0, len(x)):
            result = result + x[j] * math.log(theta_est(data_set, c, j, smooth))
        if result > max:
            max = result
            argmax = c
    return argmax
    

<h2 style="color: red;">Task:</h2>

Now let's test your implementation. The cell below has a pre-defined data set and test sample. At the moment, the smoothing parameter is set to 0 for the naïve Bayes classifier. Run the cell as-is and observe what the classifier predicts. Then, change the smoothing parameter to 1 and run the cell again. **What happens to the prediction? Why does this occur? What if we change the smoothing parameter to some integer greater than 1?** Give your answers in the raw cell below.

In [152]:
data_set = [([4, 5], 1), ([4, 0], 0), ([3, 7], 1), ([0, 8], 1), ([2, 2], 0), ([1, 3], 0), ([6, 6], 1), ([9, 5], 1), ([1, 5], 1), ([3, 2], 1), ([5, 0], 1), ([4, 6], 1), ([0, 5], 1), ([0, 9], 1), ([2, 6], 1), ([1, 6], 1), ([4, 3], 1), ([6, 3], 1), ([8, 9], 1), ([2, 5], 1), ([1, 2], 0), ([3, 9], 1), ([5, 3], 1), ([3, 5], 1), ([8, 4], 1)]
test_sample = ([9, 3], 1)
smooth = 0
pred = naive_bayes(data_set, test_sample[0], smooth)
print('Test sample:', test_sample[0])
print('True label:', test_sample[1])
print('Naive Bayes prediction (with smoothing {}): {}'.format(smooth, pred))

Test sample: [9, 3]
True label: 1
Naive Bayes prediction (with smoothing 0): 0


### Part 4: Just for Fun
---

A set of points is said to be **shattered** by the set of all linear classifiers if, for all ways of assigning labels to the points, there exists a linear classifier that will give the points the assigned labels. Thus, for example, any set of three points in $\mathbb{R}^2$ that are not colinear can be shattered by linear classifiers (see the figure below). However, all sets of four points (such as the four points from the XOR example) cannot be shattered. **What is the largest set of points that can be shattered for points in $\mathbb{R}^3$**? Provide your answer in the raw text cell below.

![](shattered_2d.png)

### Submission
---

You will only be submitting your Jupyter Notebook file, *hw5.ipynb*. Do not worry about submitting the additional files that came with the assignment. Furthermore, as a reminder, part of your grade is your documentation. Each of the functions you implemented as part of this assignment **must** be documented; failure to do so will result in a penalty.

Please upload your *hw5.ipynb* file to CMS by **Wednesday, April 17 @ 1:24pm**.